In [ ]:
# Dadi et al. (2020) DiFuMo Parcellations
# Download and Usage Guide: https://parietal-inria.github.io/DiFuMo/
# Citation: https://www.sciencedirect.com/science/article/pii/S1053811920306121


import warnings
warnings.simplefilter("ignore")
# Fetch n=10 brain images for demo
# from nilearn.datasets import fetch_localizer_contrasts

func_data = sub02.func
cmaps = func_data.cmaps
anats = func_data.anats
# Simply visualize sentence listening activations on subject-specific T1 image

import matplotlib.pyplot as plt
from nilearn import plotting

fig, axes = plt.subplots(figsize=(12, 10), ncols=2, nrows=5)
axes = axes.reshape(-1)

cut_coords = []

for i, (anat_img, cmap) in enumerate(zip(anats, cmaps)):
    display = plotting.plot_stat_map(cmap, bg_img=anat_img,
                                     figure=fig, axes=axes[i],
                                     colorbar=False)
    cut_coords.append(display.cut_coords)



from fetcher import fetch_difumo
sub02.maps_img = fetch_difumo(dimension=64).maps
from nilearn.input_data import NiftiMapsMasker

maps_masker = NiftiMapsMasker(maps_img=maps_img, verbose=1)
signals = maps_masker.fit_transform(cmaps)
print("Per ROIs signal: {0}".format(signals.shape))

# Image reconstruction
compressed_maps = maps_masker.inverse_transform(signals)
print("Each index in 4th dimension is reconstructed map: {0}".format(compressed_maps.shape))

# Visualize compressed contrast maps
from nilearn.image import iter_img


fig, axes = plt.subplots(figsize=(12, 10), ncols=2, nrows=5)
axes = axes.reshape(-1)

for i, (anat_img, cmap) in enumerate(zip(anats, iter_img(compressed_maps))):
    plotting.plot_stat_map(cmap, bg_img=anat_img, 
                           figure=fig, axes=axes[i],
                           colorbar=False, cut_coords=cut_coords[i])
    
